## Config

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [16]:
"""
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
import openai

openai.log = "None"
"""

# ServiceContext  (high level)
Service context is a container that contains the following objects that are commonly used:
- LLMPredictor (class for configuring the LLM)
- PromptHelper (For configuring input size/chucking)
- BaseEmbedding (For configuring the embedding)
- Node_parser

**PARAMETERS:**
- llm_predictor (Optional[BaseLLMPredictor]) – LLMPredictor
- prompt_helper (Optional[PromptHelper]) – PromptHelper
- embed_model (Optional[BaseEmbedding]) – BaseEmbedding or “local” (use local model)
- node_parser (Optional[NodeParser]) – NodeParser
- llama_logger (Optional[LlamaLogger]) – LlamaLogger (deprecated)
- chunk_size (Optional[int]) – chunk_size
- callback_manager (Optional[CallbackManager]) – CallbackManager
- system_prompt (Optional[str]) – System-wide prompt to be prepended to all input prompts, used to guide system “decision making”
- query_wrapper_prompt (Optional[BasePromptTemplate]) – A format to wrap passed-in input queries.

## embed_model, Embeddings

### OpenAI


PARAMETERS
mode (str) – Mode for embedding. Defaults to OpenAIEmbeddingMode.TEXT_SEARCH_MODE. Options are:

- OpenAIEmbeddingMode.SIMILARITY_MODE

- OpenAIEmbeddingMode.TEXT_SEARCH_MODE

OpenAI specifically has a "text-embedding-ada-002" model (their default) which is simply what TEXT_SEARCH_MODE selects

In [14]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index import ServiceContext

embed_model_oai= OpenAIEmbedding(
    mode="text_search",  # default, alternatively set to "similarity"
    model="text-embedding-ada-002"  # default
)

In [15]:
service_context = ServiceContext.from_defaults(
  #Default openai settings via
  #embed_model=openai
  
  #Or loading the detailed embed_model
  embed_model=embed_model_oai
)

### Hugging face

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model_hf = HuggingFaceEmbedding(
    model_name='intfloat/multilingual-e5-small',
    normalize=True,
    #Instruction to prepend query
    query_instruction='query:',
    #Instruction to prepend text
    text_instruction='passage:'
    
    #Or loading default values via
    #embed_model="local:intfloat/multilingual-e5-small"
)

In [5]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
  embed_model="local:intfloat/multilingual-e5-small"
)

## llm 

(llm_predictor function apparently about to be depreciated)

**llm classes to change LLM related parameters**
For example, to set The number of output tokens is usually set to some low number by default (for instance, with OpenAI the default is 256). Not true, just tested with default, it makes no difference!

In [44]:
from llama_index.llms import OpenAI

# define LLM
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")

resp = llm.complete('What is the meaning of life?')
print(resp)

The meaning of life is a philosophical question that has been debated by scholars, theologians, and individuals for centuries. Different perspectives and beliefs offer various interpretations of life's purpose. Some may find meaning in religious or spiritual beliefs, while others may seek fulfillment through personal relationships, achievements, or contributing to society. Ultimately, the meaning of life is subjective and can vary from person to person.


In [76]:
# define LLM
llm = OpenAI(temperature=0.0,
             model="gpt-3.5-turbo",
             max_tokens=1024)

resp = llm.complete('What is the meaning of life?')
print(resp)

The meaning of life is a philosophical question that has been debated by scholars, theologians, and individuals for centuries. Different perspectives and beliefs offer various interpretations of life's purpose. Some may find meaning in religious or spiritual beliefs, while others may seek fulfillment through personal relationships, achievements, or contributing to society. Ultimately, the meaning of life is subjective and can vary from person to person.


Add to **service context** as argument,

In [37]:
service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model= embed_model_hf)

#### **Or use LLM directly**

In [33]:
resp = OpenAI().complete('Paul Graham is ') 
print(resp)

Hello! I am an AI language model developed by OpenAI, and I don't have a personal name. You can simply refer to me as OpenAI Assistant. How can I assist you today?


In [35]:
from llama_index.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are a pirate with a colorful personality"),
    ChatMessage(role="user", content="What is your name"),
]
resp = OpenAI().chat(messages)
print(resp)

assistant: Ahoy there, matey! The name's Captain Crimsonbeard, the most colorful pirate to sail the seven seas!


## PromptHelper

Helper function to deal with LLM Context window limitations.

Calculates available context size by starting with the max context size and reserving space for the prompt template and the ouput.

ALso provides utility for 'repacking' text chunks retrieved from index to fit the LLM context window.

In [49]:
from llama_index import PromptHelper

#Default room left for output, DEFAULT_NUM_OUTPUTS = 256 (tokens)
prompt_helper = PromptHelper(num_output=256)

In [50]:
service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model= embed_model_hf,
                                                prompt_helper=prompt_helper)

## Nodeparser

In [51]:
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser

node_parser_512 = SimpleNodeParser.from_defaults(
    text_splitter=TokenTextSplitter(chunk_size=512, chunk_overlap=20)
)

## Using ServiceContext

In [54]:
service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model= embed_model_hf,
                                                prompt_helper=prompt_helper,
                                                node_parser=node_parser_512
                                                )

**Local configuration**

In [60]:
"""
query_engine = index.as_query_engine(service_context=service_context)
response = query_engine.query("What did the author do growing up?")
print(response)
"""

'\nquery_engine = index.as_query_engine(service_context=service_context)\nresponse = query_engine.query("What did the author do growing up?")\nprint(response)\n'

**Global configuration> Default pipeline for entire llamaindex pipeline**

In [59]:
from llama_index import set_global_service_context
set_global_service_context(service_context)

# Load data & create nodes

In [ ]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['./data/dagpengeguide-2023-clean.docx']).load_data()

In [ ]:
from llama_index.node_parser import SimpleNodeParser

node_parser = SimpleNodeParser.from_defaults(chunk_size=512, chunk_overlap=20)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

Parsing documents into nodes:   0%|          | 0/1 [00:00<?, ?it/s]

# Creating indexes

- Summary Index: Stores Nodes as a sequential chain, and during query time, all Nodes are loaded into the Response Synthesis module if no other query parameters are specified.
- Vector Store Index: Stores each Node and a corresponding embedding in a Vector Store, and queries involve fetching the top-k most similar Nodes.
- Tree Index: Builds a hierarchical tree from a set of Nodes, and queries involve traversing from root nodes down to leaf nodes.
- Keyword Table Index: Extracts keywords from each Node to build a mapping, and queries extract relevant keywords to fetch corresponding Nodes.

### Vectore Store Index

In [58]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex(nodes, service_context=service_context, show_progress=True)

Generating embeddings:   0%|          | 0/48 [00:00<?, ?it/s]

### Summary index
Using LLM calls to generate keywords

Not working , context window error, and seems overly complicated?

In [85]:
"""
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index import get_response_synthesizer


chatgpt = OpenAI(temperature=0.0,
             model="gpt-3.5-turbo")

prompt_helper = PromptHelper(num_output=256)

service_context = ServiceContext.from_defaults(llm=chatgpt,
                                               chunk_size=512,
                                               prompt_helper=prompt_helper,
)

response_synthesizer = get_response_synthesizer(
    response_mode="compact", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    documents=documents,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
    show_progress=True,
)"""

'\nfrom llama_index.indices.document_summary import DocumentSummaryIndex\nfrom llama_index import get_response_synthesizer\n\n\nchatgpt = OpenAI(temperature=0.0,\n             model="gpt-3.5-turbo")\n\nprompt_helper = PromptHelper(num_output=256)\n\nservice_context = ServiceContext.from_defaults(llm=chatgpt,\n                                               chunk_size=512,\n                                               prompt_helper=prompt_helper,\n)\n\nresponse_synthesizer = get_response_synthesizer(\n    response_mode="compact", use_async=True\n)\ndoc_summary_index = DocumentSummaryIndex.from_documents(\n    documents=documents,\n    service_context=service_context,\n    response_synthesizer=response_synthesizer,\n    show_progress=True,\n)'

# Storage Context 

**For vectorstores**
- Document Stores: The repositories for storing ingested documents represented as Node objects.
- Index Stores: The places where index metadata are kept.
- Vector Stores: The storages for holding embedding vectors.

In [61]:
# Persisting to disk
index.storage_context.persist(persist_dir="db/")

**Loading from disk**

In [ ]:
import os
from llama_index import StorageContext, load_index_from_storage

#Check if db exists, if it does, load from db using the below code
if os.path.exists("db/"):
    storage_context = StorageContext.from_defaults(persist_dir="db/")
    index = load_index_from_storage(storage_context)

# Query

In [86]:
query_engine = index.as_query_engine(service_context=service_context)

In [7]:
from llama_index import set_global_service_context
set_global_service_context(service_context)

In [87]:
# Query index
query_engine = index.as_query_engine(verbose=True)
response = query_engine.query("Hvordan påvirker det mine dagpenge hvis jeg arbejder deltid?")

In [88]:
response.response

'Hvis du arbejder deltid, kan det påvirke dine dagpenge. Hver time, du arbejder, sættes ind på en beskæftigelseskonto, som du kan bruge til enten at forlænge din dagpengeperiode eller gemme til du har optjent nok timer til en ny toårig dagpengeperiode. Du kan også kombinere, så du bruger en del af timerne til at forlænge din dagpengeperiode og samtidig arbejder på at opnå en ny periode. Det er vigtigt at bemærke, at timerne kun kan anvendes én gang, så hvis du bruger dem til at forlænge din nuværende periode, kan du ikke også bruge dem til at genoptjene en ny periode.'

## Inspecting prompts [new feature]

In [10]:
prompts_dict = query_engine.get_prompts()

In [13]:
prompts_dict.keys()

dict_keys(['response_synthesizer:text_qa_template', 'response_synthesizer:refine_template'])

In [20]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [21]:
from IPython.display import Markdown, display

display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

### Updating prompts

In [11]:
from llama_index.prompts import PromptTemplate

# reset
query_engine = index.as_query_engine(response_mode="tree_summarize")

# shakespeare!
new_summary_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query in the style of a Shakespeare play.\n"
    "Query: {query_str}\n"
    "Answer: "
)
new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)

## Retriver / Query engines

In [22]:
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=4,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("Hvordan påvirker det mine dagpenge hvis jeg arbejder deltid?")
print(response)

Hvis du arbejder deltid, vil det påvirke dine dagpenge. Alle former for arbejde medfører fradrag i dagpengene, både lønnet og ulønnet arbejde. Dette gælder også arbejde uden for normal arbejdstid, i weekenden og på helligdage. Hvis du får løn for flere timer, end du arbejder, skal du også oplyse om disse timer. Det kan for eksempel være forberedelse til undervisning, tillæg for helligdage, løn under sygdom og lignende. Der vil være en fast omregningssats for at beregne fradraget i dine dagpenge baseret på din indtægt.


# Q generation


In [ ]:
llm = OpenAI(temperature=0.0,
             model="gpt-3.5-turbo")

#Make a f string template with {input} 
tmlp_dagpenge = """ Nedenfor er et uddrag af en tekst. Din opgave er at stille 1 spørgsmål, hvor svaret kan findes i denne tekst.
Du må KUN stille spørgsmål, hvor svaret kan findes direkte i teksten. Returner 1 og kun 1 spørgsmål og intet andet.

TEKST: Dagpenge udbetales for en måned ad gangen med et fast timetal på 160,33 timer for fuldtidsforsikrede og på 130 timer for deltidsforsikrede medlemmer. Der kan således maksimalt udbetales dagpenge for 160,33 henholdsvis 130 timer for en måned. Udbetalingen sker bagud for en måned.
=========
SPØRGSMÅL: Hvor mange timer kan der maksimalt udbetales dagpenge for?
\n
TEKST: Udbetalingen af dagpenge sker på baggrund af et dagpengekort (månedsoplysninger), som medlemmet udfylder og indsender til a-kassen tidligst en uge før en måned udgang.
=========
SPØRGSMÅL: Hvor tidligt kan et medlem indsende dagpengekortet?
\n
TEKST: {sektion}
=========
SPØRGSMÅL:
"""


paragrah = 'Medlemmet skal efter bekendtgørelsens § 2, stk. 3 oplyse om forventede løntimer, indkomst og andre forhold af betydning for ret til dagpenge fra tidspunktet for indsendelsen af dagpengekortet og frem til månedens udgang. Der er dermed tale om, at medlemmet angiver et forventet antal arbejdstimer m.v. i de sidste dage i måneden.'

resp = llm.complete(tmlp_dagpenge.format(sektion=paragrah))
print(resp)


In [111]:
llm = OpenAI(temperature=1.0,
             model="gpt-3.5-turbo")

#Make a f string template with {input} 
tmlp_dagpenge = """ Nedenfor er et uddrag af en tekst. Din opgave er at stille 1 spørgsmål i jeg person, hvor svaret kan findes i denne tekst
Du må KUN stille spørgsmål, formuler spørgsmålet som var du en person der stiller et personligt spørgsmål til reglerne.
Svaret skal kunne findes findes direkte i teksten. Returner 1 og kun 1 spørgsmål og intet andet.

TEKST: Dagpenge udbetales for en måned ad gangen med et fast timetal på 160,33 timer for fuldtidsforsikrede og på 130 timer for deltidsforsikrede medlemmer. Der kan således maksimalt udbetales dagpenge for 160,33 henholdsvis 130 timer for en måned. Udbetalingen sker bagud for en måned.
=========
SPØRGSMÅL: Hvor mange timer kan man maksimalt få udbetalt dagepnge for?
\n
TEKST: Udbetalingen af dagpenge sker på baggrund af et dagpengekort (månedsoplysninger), som medlemmet udfylder og indsender til a-kassen tidligst en uge før en måned udgang.
=========
SPØRGSMÅL: Hvor tidligt kan jeg indsende dagpengekortet?
\n
TEKST: {sektion}
=========
SPØRGSMÅL:
"""


paragrah = 'Medlemmet skal efter bekendtgørelsens § 2, stk. 3 oplyse om forventede løntimer, indkomst og andre forhold af betydning for ret til dagpenge fra tidspunktet for indsendelsen af dagpengekortet og frem til månedens udgang. Der er dermed tale om, at medlemmet angiver et forventet antal arbejdstimer m.v. i de sidste dage i måneden.'

resp = llm.complete(tmlp_dagpenge.format(sektion=paragrah))
print(resp)


Hvornår skal jeg oplyse om de forventede løntimer og indkomst?


In [144]:
para_list = ['Medlemmet skal efter bekendtgørelsens § 2, stk. 3 oplyse om forventede løntimer, indkomst og andre forhold af betydning for ret til dagpenge fra tidspunktet for indsendelsen af dagpengekortet og frem til månedens udgang. Der er dermed tale om, at medlemmet angiver et forventet antal arbejdstimer m.v. i de sidste dage i måneden.','Efter § 4 kan medlemmet i perioden frem til samkøringstidspunktet supplere afgivne oplysninger for hele udbetalingsperioden, og a-kassen skal behandle medlemmets oplysninger i forbindelse med samkøringen. Medlemmet kan supplere afgivne oplysninger om fx antallet af arbejdstimer, som ikke fremgår af indkomstregisteret, oplysninger om afholdt ferie, sygdom og fx, at medlemmet har været afmeldt som aktivt arbejdssøgende. De supplerende oplysninger vil herefter indgå i efterreguleringen efter bekendtgørelsens § 11. Oplysninger, som a-kassen modtager fra DFDG om medlemmets ferie, sygdom og afmelding fra jobcenteret, sidestilles med oplysninger modtaget fra medlemmet.','Medlemmet har efter lovens § 65 a, stk. 3, pligt til at give a-kassen oplysning om ethvert forhold, der kan have betydning for medlemmets ret til ydelser. Medlemmet skal derfor give a-kassen de supplerende oplysninger for udbetalingsperioden og kan derfor ikke som udgangspunkt forlade sig på, at a-kassen må forventes at få oplysningerne fra andre. Modtager a-kassen oplysningerne fra DFDG, sidestilles disse dog med oplysninger modtaget fra medlemmet, jf. ovenfor.','Kommer oplysningen til a-kassen som fx en anmeldelse fra en myndighed, en henvendelse fra jobcenteret eller fra STAR’s tilsynsmyndighed, uden at medlemmet selv har henvendt sig til a-kassen om forholdet, er den supplerende oplysning ikke omfattet af bestemmelsen.']

In [145]:
para_list2 = ['Når et ansættelsesforhold er afsluttet, indberetter arbejdsgiveren oplysninger til indkomstregisteret om feriegodtgørelse, overarbejde m.v., som relaterer sig til det afsluttede ansættelsesforhold. Oplysningerne indberettes i mange tilfælde i måneden efter, at ansættelsesforholdet er afsluttet, men hvor der i måneden i øvrigt ikke er andre forhold, som kan have betydning for udbetalingen af dagpenge. For perioden efter arbejdsophøret vil medlemmet oplyse om 0 (nul) timer. I den periode vil arbejdsgiveren ofte foretage den nævnte slutindberetning med det resultat, at sagen falder ud til kontrol blot for at konstatere, at arbejdsgiverens indberetning vedrører en periode forud for arbejdsophøret. Bestemmelsen omfatter også efterlønsmodtagere i den første måned efter overgangen.','Denne undtagelse afgrænses til kun at gælde én gang pr. dagpengeperiode/efterlønsperiode og kun for den første måned (indplaceringsmåneden henholdsvis overgangsmåneden). Hvis medlemmet opnår ny ansættelse hos den samme arbejdsgiver i indplaceringsperioden og herefter ophører anden gang hos denne arbejdsgiver, vil der som hidtil blive foretaget en månedskontrol. Dette vil særligt være relevant ved vikararbejde.', 'Hvis der i ophørsmåneden også foreligger indberetninger for en anden arbejdsgiver foretages en fuld månedskontrol i forhold til denne anden arbejdsgiver. Undtagelsen omfatter således kun i forhold til den arbejdsgiver, medlemmet er ophørt med at arbejde for.', 'Bestemmelsen omfatter ikke situationer, hvor der er tale om en ny arbejdsgiver eller ved arbejdsophør i udbetalingsperioden. I de tilfælde skal der fortsat foretages en samkøring efter stk. 1.']

In [149]:
para_list3 = ['§ 10 finder anvendelse i den måned, hvor medlemmet opbruger de 481 timer med et beskæftigelsestillæg efter lovens § 48, stk. 8, eller de 481 timer med 71,5 % af højeste dagpenge til dimittender efter lovens § 49, stk. 5, 1. og 2. pkt. I den måned sker udbetalingen af dagpenge med mere end en sats.','Ved regulering efter bestemmelsen forstås den proces, hvor a-kassen efter samkøring, jf. lovens § 46, stk. 6, genberegner antallet af timer for måneden. Bestemmelsen dækker såvel reguleringer som følge af lovens § 85 e og senere ændringer som følge af en a-kasses afgørelse om tilbagebetaling og sanktion, jf. lovens § 86.','Reguleringen af måneden med mere end én sats sker ved, at der først sker regulering i de timer, der er udbetalt med den senest anvendte sats. Den senest anvendte sats er den sats, der er gældende på udbetalingstidspunktet. Først hvis den samlede regulering for måneden overstiger det antal timer, som der er udbetalt med den senest anvendte sats, skal der også ske regulering i timer udbetalt med den tidligere anvendte sats.','Det afgørende for, om der skal ske regulering i timer med den ene eller begge satser er, om det samlede antal timer, der skal reguleres for, overstiger antallet af timer udbetalt med den senest anvendte sats. Hvis det samlede antal timer, der skal reguleres for, overstiger antallet af timer udbetalt med den senest anvendte sats betyder det, at den tidligere anvendte sats alligevel ikke var opbrugt, hvorfor der også skal ske regulering i disse timer.']

In [151]:
para_list4 = ['A-kassen skal i forbindelse med den udbetaling, der foretages den sidste bankdag i måneden, sende medlemmet en udbetalingsspecifikation med klagevejledning, jf. § 23, stk. 1, i bekendtgørelse om kravene til a-kassernes sagsbehandling og afgørelse. I de tilfælde hvor a-kassen har sendt medlemmet en meddelelse om en forventet efterregulering, som medlemmet ikke har reageret på, kan udbetalingsspecifikationen henvise til meddelelsen om den forventede efterregulering.','Hvis efterreguleringen af ydelser kun delvist kan dække efterreguleringen, skal a-kassen i forbindelse med udbetalingsspecifikationen oplyse medlemmet om, hvor mange timer, der er til "rest" og at medlemmet skal betale et beløb svarende til reguleringen, såfremt reguleringen ikke kan gennemføres inden for 3 måneder, jf. § 23, stk. 2 i bekendtgørelse om kravene til a-kassernes sagsbehandling og afgørelser.','Hvis en efterregulering ikke kan foretages i den førstkommende udbetaling af ydelser, fordi der ikke udbetales ydelser, skal a-kassen sende medlemmet en meddelelse med angivelse af det antal timer, medlemmet skulle have været efterreguleret for. Meddelelsen skal oplyse om reguleringens størrelse og at medlemmet skal tilbagebetale et beløb svarende til reguleringen, såfremt reguleringen ikke kan gennemføres inden for 3 måneder, jf. § 23, stk. 2, i bekendtgørelse om kravene til a-kassernes sagsbehandling og afgørelser.','En udbetaling er at betragte som endelig, selvom efterreguleringen konkret ikke kan gennemføres, fordi medlemmet ikke modtager en ydelse i måneden, eller ydelsen ikke er tilstrækkelig til at dække det beløb, medlemmet skal reguleres for.']

In [152]:
#Para_list contains a list of strings, count the total number of items (that is, 1 string = 1 item) in the list:
print(len(para_list4))

4


In [113]:
resp_list = []
for pararaph in para_list:
    resp = llm.complete(tmlp_dagpenge.format(sektion=pararaph))
    resp_list.append(resp)

In [118]:
resp_list2 = []
for pararaph in para_list2:
    resp = llm.complete(tmlp_dagpenge.format(sektion=pararaph))
    resp_list2.append(resp)

In [153]:
resp_list3 = []
for pararaph in para_list3:
    resp = llm.complete(tmlp_dagpenge.format(sektion=pararaph))
    resp_list3.append(resp)

In [154]:
resp_list4 = []
for pararaph in para_list4:
    resp = llm.complete(tmlp_dagpenge.format(sektion=pararaph))
    resp_list4.append(resp)

In [114]:
for i, resp in enumerate(resp_list):
    print(f'Teskst: {para_list[i]}  \n')
    print(f'Spørgsmål: {resp}')
    print('\n')

Teskst: Medlemmet skal efter bekendtgørelsens § 2, stk. 3 oplyse om forventede løntimer, indkomst og andre forhold af betydning for ret til dagpenge fra tidspunktet for indsendelsen af dagpengekortet og frem til månedens udgang. Der er dermed tale om, at medlemmet angiver et forventet antal arbejdstimer m.v. i de sidste dage i måneden.  

Spørgsmål: Hvad skal medlemmet oplyse om i dagpengekortet?


Teskst: Efter § 4 kan medlemmet i perioden frem til samkøringstidspunktet supplere afgivne oplysninger for hele udbetalingsperioden, og a-kassen skal behandle medlemmets oplysninger i forbindelse med samkøringen. Medlemmet kan supplere afgivne oplysninger om fx antallet af arbejdstimer, som ikke fremgår af indkomstregisteret, oplysninger om afholdt ferie, sygdom og fx, at medlemmet har været afmeldt som aktivt arbejdssøgende. De supplerende oplysninger vil herefter indgå i efterreguleringen efter bekendtgørelsens § 11. Oplysninger, som a-kassen modtager fra DFDG om medlemmets ferie, sygdom o

In [142]:
for i, resp in enumerate(resp_list2):
    print(f'Teskst: {para_list2[i]}  \n')
    print(f'Spørgsmål: {resp}')
    print('\n')

Teskst: Når et ansættelsesforhold er afsluttet, indberetter arbejdsgiveren oplysninger til indkomstregisteret om feriegodtgørelse, overarbejde m.v., som relaterer sig til det afsluttede ansættelsesforhold. Oplysningerne indberettes i mange tilfælde i måneden efter, at ansættelsesforholdet er afsluttet, men hvor der i måneden i øvrigt ikke er andre forhold, som kan have betydning for udbetalingen af dagpenge. For perioden efter arbejdsophøret vil medlemmet oplyse om 0 (nul) timer. I den periode vil arbejdsgiveren ofte foretage den nævnte slutindberetning med det resultat, at sagen falder ud til kontrol blot for at konstatere, at arbejdsgiverens indberetning vedrører en periode forud for arbejdsophøret. Bestemmelsen omfatter også efterlønsmodtagere i den første måned efter overgangen.  

Spørgsmål: Hvordan indberettes oplysninger om feriegodtgørelse og overarbejde efter et ansættelsesforhold er afsluttet?


Teskst: Denne undtagelse afgrænses til kun at gælde én gang pr. dagpengeperiode/e

In [124]:
llm = OpenAI(temperature=1.0,
             model="gpt-3.5-turbo")

#Make a f string template with {input} 
tmlp_check = """ Nedenfor er et uddrag af en tekst og et tilhørende spørgsmål. Din opgave er at vurdere om svaret på spørgsmålet kan findes i teksten.
Du må KUN svare TRUE eller FALSE. Returner 1 og kun 1 svar og intet andet.

TEKST: Dagpenge udbetales for en måned ad gangen med et fast timetal på 160,33 timer for fuldtidsforsikrede og på 130 timer for deltidsforsikrede medlemmer. Der kan således maksimalt udbetales dagpenge for 160,33 henholdsvis 130 timer for en måned. Udbetalingen sker bagud for en måned.
SPØRGSMÅL: Hvor mange timer kan man maksimalt få udbetalt dagepnge for?
=========
SVAR: TRUE
\n
TEKST: Hvis der i ophørsmåneden også foreligger indberetninger for en anden arbejdsgiver foretages en fuld månedskontrol i forhold til denne anden arbejdsgiver. Undtagelsen omfatter således kun i forhold til den arbejdsgiver, medlemmet er ophørt med at arbejde for.
SPØRGSMÅL: Hvad skal der gøres i tilfælde af en ny arbejdsgiver eller arbejdsophør i udbetalingsperioden?
=========
SVAR: FALSE
\n
TEKST: {sektion}
SPØRGSMÅL: {spørgsmål}
=========
SVAR:
"""

In [ ]:
#Mixed list

In [146]:
for i, question in enumerate(resp_list):
    print(question)
    resp = llm.complete(tmlp_check.format(sektion=para_list[i], spørgsmål=question))
    print(resp)

Hvad skal medlemmet oplyse om i dagpengekortet?
TRUE
Hvilke oplysninger kan medlemmet supplere a-kassen med?
TRUE
Skal jeg selv give a-kassen supplerende oplysninger for udbetalingsperioden?
TRUE
Hvilke oplysninger er ikke omfattet af bestemmelsen, hvis de kommer fra en myndighed, jobcenteret eller STAR's tilsynsmyndighed uden at medlemmet selv har henvendt sig til a-kassen?
TRUE


In [155]:
for i, question in enumerate(resp_list3):
    print(f'Teskst: {para_list4[i]}  \n')
    print(f'Spørgsmål: {question}')
    resp = llm.complete(tmlp_check.format(sektion=para_list4[i], spørgsmål=question))
    print(resp)
    print('\n')

Teskst: A-kassen skal i forbindelse med den udbetaling, der foretages den sidste bankdag i måneden, sende medlemmet en udbetalingsspecifikation med klagevejledning, jf. § 23, stk. 1, i bekendtgørelse om kravene til a-kassernes sagsbehandling og afgørelse. I de tilfælde hvor a-kassen har sendt medlemmet en meddelelse om en forventet efterregulering, som medlemmet ikke har reageret på, kan udbetalingsspecifikationen henvise til meddelelsen om den forventede efterregulering.  

Spørgsmål: Hvad sker der i den måned, hvor medlemmet opbruger 481 timer med et beskæftigelsestillæg eller 481 timer med 71,5 % af højeste dagpenge til dimittender?
FALSE


Teskst: Hvis efterreguleringen af ydelser kun delvist kan dække efterreguleringen, skal a-kassen i forbindelse med udbetalingsspecifikationen oplyse medlemmet om, hvor mange timer, der er til "rest" og at medlemmet skal betale et beløb svarende til reguleringen, såfremt reguleringen ikke kan gennemføres inden for 3 måneder, jf. § 23, stk. 2 i bek